# Match Steam Banners

Code inspired from https://github.com/woctezuma/match-steam-banners

## Setting

### Clone my repository

In [ ]:
%cd /content/

In [ ]:
!rm -rf match-steam-banners/

!git clone https://github.com/woctezuma/match-steam-banners.git

### Install Python requirements

In [ ]:
%cd /content/match-steam-banners/

In [ ]:
!pip install -r requirements.txt -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
%pip install git+https://github.com/openai/CLIP.git

## Image data

### Mount Google Drive

In [ ]:
!pip install Google-Colab-Transfer

In [ ]:
import colab_transfer

colab_transfer.mount_google_drive()

### Import image data from Google Drive

In [ ]:
colab_transfer.copy_file('resized_vertical_steam_banners_224.tar',
                         source='/content/drive/MyDrive/data/',
                         destination='/content/match-steam-banners/data/')

# Alternatively, run:
# !gdown --id 1--cxY3jvTVWq-lZt8NvfN2fHND7YhKN4
# %mkdir -p data/
# %mv resized_vertical_steam_banners_224.tar data/

In [ ]:
%cd /content/match-steam-banners/
!tar -xf data/resized_vertical_steam_banners_224.tar

In [ ]:
%cd /content/match-steam-banners/
%mv data/resized_vertical_steam_banners_224 data/resized_vertical_steam_banners

## 1. Features

First, compute and store the 1024 features corresponding to each banner.

### Compute features

It takes about 1 minute to process 1k images. Total time: ~ 20 minutes.

In [ ]:
!python build_feature_index.py

### Export feature data to Google Drive

In [ ]:
# colab_transfer.copy_file('frozen_app_ids.txt',
#                          source='/content/match-steam-banners/data/',
#                          destination=colab_transfer.get_path_to_home_of_google_drive())

# colab_transfer.copy_file('label_database.avg.npy', 
#                          source='/content/match-steam-banners/data/', 
#                          destination=colab_transfer.get_path_to_home_of_google_drive())

### Import feature data from Google Drive

In [ ]:
colab_transfer.copy_file('frozen_app_ids.txt',
                         source=colab_transfer.get_path_to_home_of_google_drive(),
                         destination='/content/match-steam-banners/data/')

colab_transfer.copy_file('label_database.avg.npy', 
                         source=colab_transfer.get_path_to_home_of_google_drive(),
                         destination='/content/match-steam-banners/data/')

In [ ]:
# Alternatively:

from pathlib import Path

%mkdir -p data

if not Path('data/frozen_app_ids.txt').exists():
  print('Downloading')
  !gdown --id 1YP6rG36wYSZMoXWL1acamRw6_R9l5TUA
  %mv frozen_app_ids.txt data/

if not Path('data/label_database.avg.npy').exists():
  print('Downloading')
  !gdown --id 10nb4TPhGsy7YvpkAuULUmBCkzm4DAm36
  %mv label_database.avg.npy data/

## 2. Similar games

Find the 10 most similar store banners to curated query appIDs.

### Ping SteamSpy and GameDataCrunch API

SteamSpy API may block requests from Google Colab. In this case, responses are empty:

> JSONDecodeError: Expecting value: line 1 column 1 (char 0)

If you encounter this issue, you can bypass it by running the following from **your own local machine** to ping SteamSpy API:

In [ ]:
!python steam_spy_utils.py

Then **manually** upload the 2 cached files from within Colab's interface:
```
data/
├ 20200823_gamedatacrunch.json   # GameDataCrunch database of games
└ 20200823_top_100_app_ids.txt   # top100 most played games in the past 2 weeks
```

The GameDataCrunch database is used to retrieve game names corresponding to appIDs.

Top 100 appIDs are used as a list of popular games to test our algorithm onto.

### Run the workflow

In [ ]:
!python retrieve_similar_features.py

## 3. Unique games

Find the one most similar store banner to all appIDs available on the store, then display the most unique games.

In [ ]:
!python find_unique_games.py